In [3]:
import requests
from lxml import html
from bs4 import BeautifulSoup as bs

In [85]:
sport_dict = {
         '田徑'   : 'A',
         '足球'   : 'B',
         '籃球'   : 'C',
         '排球'   : 'D',
         '網球'   : 'E',
         '棒球'   : 'F',
         '壘球'   : 'G',
         '橄欖球' : 'H',
         '羽球'   : 'S',
         '桌球'   : 'T'
    }
volley_court_dict = {
     '00':'光復低場排球1',
     '01':'光復低場排球1',
     '02':'光復低場排球1',
     '10':'光復低場排球2',
     '11':'光復低場排球2',
     '12':'光復低場排球2',
     '20':'光復低場排球3',
     '21':'光復低場排球3',
     '22':'光復低場排球3',
     '30':'光復低場排球4',
     '31':'光復低場排球4',
     '32':'光復低場排球4',
     '40':'光復低場排球5',
     '41':'光復低場排球5',
     '42':'光復低場排球5',
     '50':'光復高場排球東',
     '51':'光復高場排球東',
     '52':'光復高場排球東',
     '60':'光復高場排球西',
     '61':'光復高場排球西',
     '62':'光復高場排球西',
}
time_dict = {
     '0':'0600~1200',
     '1':'1200~1800',
     '2':'1800~2100'
}

In [49]:
def get_sport_raw(date:str, sport_in:str):
    session = requests.session()
    url = 'https://cet.acad.ncku.edu.tw/ste/index.php?c=ste11211'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
        # 'Cookie': 'PHPSESSID=1aea228467450d51917fc7f8f835ff86',
        'Cookie': 'PHPSESSID=8f4bace3465bd821988e2ed78be708b8'
        # TODO: 把這邊的 Cookie 改成 CookieJar
        # 'X-Requested-With': 'XMLHttpRequest'
    }

    payload = {
        'sport': sport_dict[sport_in],
        'sdate': date,
        'c': 'ste11211',
        'm': 'read'
    }


    volleyball = session.post(url=url, headers=headers, data=payload)
    # print(session.cookies.get)
    # print(login.cookies, login.cookies.get_dict())
    # print(headers['Cookie'])

    # if login.cookies.get_dict():        #保持cookie有效
        # login.cookies.update(login.cookies)
    # volleyball = session.post(url=url_get, headers=headers, data=payload)
    # print(login.cookies, bool(login.cookies.get_dict()))
    # print(login.text)
    # print(volleyball.text)
    # with open('test.html', 'w', encoding='utf-8') as fp:
    #     fp.write(volleyball.text)

    # replace is for '體育室 <br/>(數學系)' (in raw)
    # but in '體育室 <br>(數學系)' (in html)
    return bs(volleyball.text.replace('<br>', ''), 'html.parser')

In [99]:
# get_sport_raw('20201222', '排球')

<div class="site_table_div">
<h2>
		2020年12月22日 	</h2>
<table class="gray_table2">
<thead>
<tr>
<!-- 顯示欄位標題 -->
<th style="width:10%;"></th>
<th style="width:5%;">06:0007:00</th><th style="width:5%;">07:0008:00</th><th style="width:5%;">08:0009:00</th><th style="width:5%;">09:0010:00</th><th style="width:5%;">10:0011:00</th><th style="width:5%;">11:0012:00</th><th style="width:5%;">12:0013:00</th><th style="width:5%;">13:0014:00</th><th style="width:5%;">14:0015:00</th><th style="width:5%;">15:0016:00</th><th style="width:5%;">16:0017:00</th><th style="width:5%;">17:0018:00</th><th style="width:5%;">18:0019:00</th><th style="width:5%;">19:0020:00</th><th style="width:5%;">20:0021:00</th>
</tr>
</thead>
<tbody>
<!-- 顯示資料 -->
<tr><th>光復低場排球1D1001</th><td class="cell3" colspan="6">H2統計系</td><td class="cell4" colspan="6">體育室  </td><td class="cell3" colspan="3">Q1微電所</td></tr><tr><th>光復低場排球2D1002</th><td colspan="6"></td><td class="cell4" colspan="6">體育室  </td><td class="cell3" colspan="3">

In [65]:
def retreive_form(soup):
    if not isinstance(soup, type(bs())):
        raise TypeError("soup must be a " + repr(type(bs())))
    else:
        form =[]
        row = []
        counter = 0
        for content in soup.find_all('td'):
            # print(content, '\t\t', repr(content.string))
            if content.string not in ('\xa0\xa0', '可借用', '不可借用', '已借用'):
                if counter == 3:
                    counter = 0
                    form.append(row)
                    row = []
                row.append(content.string)
                counter += 1
    
    return form

In [43]:
from datetime import datetime, timedelta
def gen_dates(start:str, end:str):
    start_date = datetime.strptime(start, '%Y%m%d')
    end_date = datetime.strptime(end, '%Y%m%d')
    serial = []
    if (end_date - start_date).days >= 0:
        for i in range((end_date - start_date).days+1):
            tmp = start_date + timedelta(days=i)
            serial.append('{0:>{3}4}{1:>{3}2}{2:>{3}2}'.format(tmp.year, tmp.month, tmp.day, 0))
            
    return serial

In [8]:
# output to html file
with open('test1.html', 'w', encoding='utf-8') as fp:
    fp.write(soup.prettify())

NameError: name 'soup' is not defined

In [51]:
new_s = get_sport_raw('20201221', '排球')
t = retreive_form(new_s)
t
# print(t)
# print(type(new_s))
# new_s = new_s.replace('<br/', '')
# print(repr(new_s))

[['F7資訊系', '體育室  ', 'I6物治系'],
 ['H1會計系', '體育室  ', 'B2外文系'],
 ['F1系統系', '體育室  ', '體育室  (數學系)'],
 [None, '體育室  ', 'I5醫學系'],
 ['F2都計系', '體育室  ', 'K5台文所'],
 ['F6測量系', '體育室  ', '體育室  (貴重儀器中心)'],
 ['P8醫工所', '體育室  ', 'I8藥學系']]

In [155]:
seq = gen_dates('20201224', '20201231')
def get_mapped_form(time_seq:list):
    situation = {}
    for dates in time_seq:
        print(dates)
        m = get_sport_raw(dates, '排球')
        situation[dates] = retreive_form(m)
        # print(situation)

In [147]:
# find empty time
def find_free_time(seq:list, gender:str='a'):
    # gender = 'a' -> both gender
    free = []
    for dates in seq:
        for row in range(7):
            for column in range(3):
                # print(type(situation[dates][row][column]))
                if isinstance(situation[dates][row][column], type(None)):
                    element = '{0} {1} {2}'.format(dates, time_dict[str(column)], court_dict[str(row*10+column).zfill(2)])
                    if gender.lower() == 'b': # boy
                        if row in [3, 4, 5]:
                            continue
                    elif gender.lower() == 'g': # girl
                        if row not in [3, 4, 5]:
                            continue
                    free.append(element)
    return free
                    

In [153]:
# format outcome
free = find_free_time(seq, 'g')
def format_free_time(free:list):
    new_free = []
    if len(free) != 1:
        old = -1
        for lines in free:
            if old == -1:
                old = int(lines[4:8])
            elif old != int(lines[4:8]):
                old = int(lines[4:8])
                new_free.append('-' * (len(lines)+6))
            new_free.append(lines)
    
    return new_free

In [154]:
for content in format_free_time(free):
    print(content)

20201228 0600~1200 光復低場排球4
20201228 0600~1200 光復低場排球5
20201228 0600~1200 光復高場排球東
--------------------------------
20201229 0600~1200 光復低場排球4
20201229 0600~1200 光復低場排球5
20201229 0600~1200 光復高場排球東
--------------------------------
20201230 0600~1200 光復低場排球4
20201230 0600~1200 光復低場排球5
20201230 1800~2100 光復低場排球5
20201230 0600~1200 光復高場排球東
--------------------------------
20201231 0600~1200 光復低場排球4
20201231 1800~2100 光復低場排球4
20201231 0600~1200 光復低場排球5
20201231 0600~1200 光復高場排球東


In [256]:
soup = bs(result.text, 'html.parser')
# soup

In [405]:
class a:
    value = 3

In [407]:
def add(t):
    print('before =', t.value)
    t.value = 0
    print('after =', t.value)
    return t 